In [45]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import collections

In [2]:
csv_files = list(Path('submissions').glob('submission-*.csv'))

In [19]:
csvs = []
for f in csv_files:
    csv = pd.read_csv(f)
    csvs.append(csv)

In [26]:
id = []
for _, row in csvs[0].iterrows():
    id.append(row['id'])

In [41]:
digits = []
for csv in csvs:
    digit = []
    for _, row in csv.iterrows():
        digit.append(row['digit'])
    digits.append(digit)

In [ ]:
def best_one(s):
    

In [49]:
new_digit = []
for i in range(len(digits[0])):
    s = []
    for j in range(len(digits)):
        s.append(digits[j][i])
    
    counter = collections.Counter(s)
    max_v, max_k = 0, 0
    for k in counter.keys():
        if max_v < counter[k]:
            max_v = counter[k]
            max_k = k
    new_digit.append(max_k)

In [52]:
new_csv = pd.DataFrame({'id': id, 'digit': new_digit})

In [54]:
new_csv.to_csv('submissions/ensemble-0119-2147-2150-2157-2158.csv', index=None)